In [3]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 4.6 MB/s eta 0:00:00a 0:00:01m


In [2]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt

# Define the path to the data directory
data_path = "chb-mit-scalp-eeg-database-1.0.0/chb01"

# Function to preprocess EEG data and plot seizures
def preprocess_and_plot(file_path, seizures_info):
    # Load EEG data using MNE
    raw = mne.io.read_raw_edf(file_path, preload=True)

    # Plot the raw EEG data
    raw.plot()
    raw.notch_filter([60, 120], fir_design='firwin')
    raw.resample(128, npad='auto')
    raw.plot()
    if seizures_info:
        for onset, duration in seizures_info:
            raw.plot(start=onset, duration=duration, color='r')

    return raw

# Process each line in the summary file
with open(os.path.join(data_path, "chb01-summary.txt"), 'r') as summary_file:
    file_info = []
    for line in summary_file:
        line = line.strip()

        if not line:
            continue

        # If the line contains "File Name", it's the start of a new file entry
        if line.startswith("File Name"):
            # Process the previous file info, if any
            if file_info:
                try:
                    file_name = file_info[0].split(":")[1].strip()
                    num_seizures = int(file_info[-1].split(":")[1].strip())
                    if num_seizures > 0:
                        seizures_info = [(int(file_info[i+1].split(":")[1].strip().split()[0]),
                                          int(file_info[i+2].split(":")[1].strip().split()[0]))
                                         for i in range(1, num_seizures * 2, 2)]
                    else:
                        seizures_info = []

                    # Process the EEG data and plot seizures
                    print(f"Processing file: {file_name}")
                    file_path = os.path.join(data_path, file_name)
                    preprocess_and_plot(file_path, seizures_info)

                except ValueError as e:
                    print(f"Error processing file: {file_name}")
                    print(f"Error message: {e}")

            # Start collecting information for the new file
            file_info = [line]
        else:
            file_info.append(line)

    if file_info:
        try:
            file_name = file_info[0].split(":")[1].strip()
            num_seizures = int(file_info[-1].split(":")[1].strip())
            if num_seizures > 0:
                seizures_info = [(int(file_info[i+1].split(":")[1].strip().split()[0]),
                                  int(file_info[i+2].split(":")[1].strip().split()[0]))
                                 for i in range(1, num_seizures * 2, 2)]
            else:
                seizures_info = []

            # Process the EEG data and plot seizures
            print(f"Processing file: {file_name}")
            file_path = os.path.join(data_path, file_name)
            preprocess_and_plot(file_path, seizures_info)

        except ValueError as e:
            print(f"Error processing file: {file_name}")
            print(f"Error message: {e}")

# Show the plots
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'chb-mit-scalp-eeg-database-1.0.0/chb01/chb01-summary.txt'

In [4]:
import os
import mne
import numpy as np
from scipy import signal, stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define the path to your data folder
data_path = "chb-mit-scalp-eeg-database-1.0.0/chb01"  # Replace with your actual path

# Process each line in the summary file
for line in summary_content:
    line = line.strip()
    if not line:
        continue

    if line.startswith("File Name"):
        file_info = [line]
    else:
        file_info.append(line)

    # If the line contains "Number of Seizures", it's the end of the file entry
    if line.startswith("Number of Seizures"):
        file_name = file_info[0].split(":")[1].strip()
        start_time = file_info[1].split(":")[1].strip()
        end_time = file_info[2].split(":")[1].strip()
        num_seizures = int(file_info[-1].split(":")[1].strip())

        edf_file_path = os.path.join(data_path, file_name)
        raw = mne.io.read_raw_edf(edf_file_path, preload=True)
        data, _ = raw[:, :]  # Get EEG data as a NumPy array

        # Feature extraction
        # Time-domain features
        mean_values = np.mean(data, axis=1)
        std_values = np.std(data, axis=1)

        # Frequency-domain features
        # Compute power spectral density (PSD)
        frequencies, psd = signal.welch(data, fs=raw.info['sfreq'], nperseg=256)

        # Compute spectral entropy using the scipy library
        spectral_entropy_value = stats.entropy(psd.T)

        # Print or store the features as needed
        print(f"File Name: {file_name}")
        print(f"Time-Domain Features: Mean={mean_values}, Std={std_values}")
        print(f"Frequency-Domain Features: Spectral Entropy={spectral_entropy_value}")
        print("------------------------")

        file_info = []

NameError: name 'summary_content' is not defined

In [ ]:
import os
import mne
import numpy as np
from scipy import signal, stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data_path = "chb-mit-scalp-eeg-database-1.0.0/chb01" 

# Initialize lists to store features and labels
all_features = []
all_labels = []

# Process each line in the summary file
for line in summary_content:
    line = line.strip()

    if not line:
        continue

    if line.startswith("File Name"):
        file_info = [line]
    else:
        file_info.append(line)
    if line.startswith("Number of Seizures"):
        # Extract relevant information
        file_name = file_info[0].split(":")[1].strip()
        start_time = file_info[1].split(":")[1].strip()
        end_time = file_info[2].split(":")[1].strip()
        num_seizures = int(file_info[-1].split(":")[1].strip())

        # Load EEG data using MNE
        edf_file_path = os.path.join(data_path, file_name)
        raw = mne.io.read_raw_edf(edf_file_path, preload=True)
        data, _ = raw[:, :]  # Get EEG data as a NumPy array

        # Feature extraction
        # Time-domain features
        mean_values = np.mean(data, axis=1)
        std_values = np.std(data, axis=1)

        # Frequency-domain features
        # Compute power spectral density (PSD)
        frequencies, psd = signal.welch(data, fs=raw.info['sfreq'], nperseg=256)

        # Compute spectral entropy using the scipy library
        spectral_entropy_value = stats.entropy(psd.T)

        # Combine features into a single feature vector
        features = np.concatenate([mean_values, std_values, spectral_entropy_value])

        # Determine the label based on the presence of seizures
        label = 1 if num_seizures > 0 else 0

        # Store features and labels
        all_features.append(features)
        all_labels.append(label)

        # Clear the file_info list for the next iteration
        file_info = []

# Convert lists to NumPy arrays
X = np.array(all_features)
y = np.array(all_labels)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
import os
import mne
import numpy as np
from scipy import signal, stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Define the path to your data folder
data_path = "chb-mit-scalp-eeg-database-1.0.0/chb01" 

# Initialize lists to store features and labels
all_features = []
all_labels = []

# Process each line in the summary file
for line in summary_content:
    line = line.strip()

    # Skip empty lines
    if not line:
        continue

    # If the line contains "File Name", it's the start of a new file entry
    if line.startswith("File Name"):
        file_info = [line]
    else:
        file_info.append(line)

    # If the line contains "Number of Seizures", it's the end of the file entry
    if line.startswith("Number of Seizures"):
        # Extract relevant information
        file_name = file_info[0].split(":")[1].strip()
        start_time = file_info[1].split(":")[1].strip()
        end_time = file_info[2].split(":")[1].strip()
        num_seizures = int(file_info[-1].split(":")[1].strip())

        # Load EEG data using MNE
        edf_file_path = os.path.join(data_path, file_name)
        raw = mne.io.read_raw_edf(edf_file_path, preload=True)
        data, _ = raw[:, :]  # Get EEG data as a NumPy array

        # Feature extraction
        # Time-domain features
        mean_values = np.mean(data, axis=1)
        std_values = np.std(data, axis=1)

        # Frequency-domain features
        # Compute power spectral density (PSD)
        frequencies, psd = signal.welch(data, fs=raw.info['sfreq'], nperseg=256)

        # Compute spectral entropy using the scipy library
        spectral_entropy_value = stats.entropy(psd.T)

        # Combine features into a single feature vector
        features = np.concatenate([mean_values, std_values, spectral_entropy_value])

        # Determine the label based on the presence of seizures
        label = 1 if num_seizures > 0 else 0

        # Store features and labels
        all_features.append(features)
        all_labels.append(label)

        # Clear the file_info list for the next iteration
        file_info = []


# Convert lists to NumPy arrays
X = np.array(all_features)
y = np.array(all_labels)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Reshape the data to 3D (number of samples, number of features, 1)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the CNN model
model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification, use 'softmax' for multi-class

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Implement dropout and early stopping
dropout_rate = 0.5
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val),
                    callbacks=[early_stopping, model_checkpoint])

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Load the best model based on validation loss
best_model = load_model('best_model.h5')

# Evaluate the best model on the test set
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [ ]:
raw.times.shape

In [ ]:
raw._data.shape

In [ ]:
import matplotlib.pyplot as plt

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Choose a random sample from the test set
sample_index = np.random.randint(0, len(X_test))

# Visualize EEG data
plt.figure(figsize=(12, 6))
plt.plot(X_test[sample_index].squeeze(), label='Original EEG Data')
plt.title('EEG Data')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

# Visualize model predictions
plt.figure(figsize=(12, 6))
plt.plot(y_pred[sample_index], label='Model Predictions', marker='o')
plt.plot(y_test[sample_index], label='True Labels', linestyle='dashed', marker='o')
plt.title('Model Predictions vs True Labels')
plt.xlabel('Time')
plt.ylabel('Class Probability')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

best_model = load_model('best_model.h5')

# Extract a segment of EEG data and corresponding time points
start_sec, end_sec = 10, 50  
sfreq = raw.info['sfreq']
start_point, end_point = int(start_sec * sfreq), int(end_sec * sfreq)

# Extract EEG data
eeg_data = raw.get_data(start=start_point, stop=end_point)

# Plot EEG data
plt.figure(figsize=(12, 6))
plt.plot(raw.times[start_point:end_point], eeg_data.T)
plt.title('EEG Data Segment')
plt.xlabel('Time (s)')
plt.ylabel('EEG Signal')
plt.show()

# Plot model predictions alongside time points
time_points = np.linspace(start_sec, end_sec, len(y_pred))

plt.figure(figsize=(12, 3))
plt.plot(time_points, y_pred, label='Model Predictions', marker='o', linestyle='-', color='blue')
plt.title('Model Predictions')
plt.xlabel('Time (s)')
plt.ylabel('Prediction Probability')
plt.legend()
plt.show()
